In [21]:
import os
import glob
import numpy as np
import tensorflow as tf
import keras
from PIL import Image
import pandas as pd

# -------- custom function (nếu model có dùng) --------


def scaling(x, scale=0.10):
    return x * scale

emb_model = keras.saving.load_model(
    "hf://logasja/FaceNet512",
    custom_objects={"scaling": scaling},
    compile=False,
    safe_mode=False
)
print("Loaded OK:", emb_model.output_shape)

print("Loaded embedding model OK")


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 24244.53it/s]


Loaded OK: (None, 512)
Loaded embedding model OK


In [9]:

# -------- utils --------
def load_face(path, size=(160, 160)):
    img = Image.open(path).convert("RGB").resize(size)
    x = np.array(img).astype("float32")
    x = (x / 127.5) - 1.0   
    return x

def l2n(x):
    return tf.math.l2_normalize(x, axis=-1)

def embed_one(image_path):
    x = load_face(image_path)[None, ...]  # (1,160,160,3)
    e = emb_model(x, training=False)
    e = l2n(e)

    if isinstance(e, tf.Tensor):
        e = e.numpy()

    e = e.reshape(-1)
    assert e.shape == (512,), f"Bad embedding shape {e.shape}"
    return e

# -------- scan data_test folder --------
DATA_DIR = "/home/tuanh/projet/face_Recognition/data_test/data"

image_paths = sorted(
    glob.glob(os.path.join(DATA_DIR, "*"))
)

records = []

for path in image_paths:
    try:
        name = os.path.splitext(os.path.basename(path))[0]  
        emb = embed_one(path)

        records.append({
            "name": name,
            "val_embedding": emb.tolist()
        })

        print(f"✓ Embedded {name}")

    except Exception as e:
        print(f"✗ Skip {path}: {e}")

df = pd.DataFrame(records)
df.to_csv("base.csv", index=False)

print(f"\nSaved base.csv with {len(df)} identities")
print(df.head())


✓ Embedded 6
✓ Embedded TuAnh
✓ Embedded pig

Saved base.csv with 3 identities
    name                                      val_embedding
0      6  [0.050949838012456894, 0.019697386771440506, 0...
1  TuAnh  [0.028406303375959396, -0.030795753002166748, ...
2    pig  [-0.01214636955410242, 0.004847864620387554, -...


In [ ]:
import pandas as pd


def load_face(path):
    img = Image.open(path).convert("RGB").resize((160,160))
    x = np.array(img).astype("float32")
    x = (x / 127.5) - 1.0   
    return x


def l2n(x): 
    return tf.math.l2_normalize(x, axis=-1)

def cos(a,b):
    return tf.reduce_sum(a*b, axis=-1)

x3 = load_face("/home/tuanh/projet/face_Recognition/data_test/data/TuAnh.jpeg")[None, ...]
x4 = load_face("/home/tuanh/projet/face_Recognition/data_test/data/TuAnh2.jpeg")[None, ...]


e3 = emb_model(x3, training=False)
e3 = l2n(e3)

e4 = emb_model(x4, training=False)
e4 = l2n(e4)

print("same cosine:", cos(e3,e4).numpy())



FileNotFoundError: [Errno 2] No such file or directory: '/home/tuanh/projet/face_Recognition/data_test/data/Tuanh.jpeg'

In [7]:
import torch
YOLOV5_WEIGHTS = "/home/tuanh/projet/face_Recognition/best.pt"

det = torch.hub.load(
    "ultralytics/yolov5",
    "custom",
    path=YOLOV5_WEIGHTS,
    force_reload=False
)
det.conf = 0.35
det.iou = 0.45

Using cache found in /home/tuanh/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2026-1-13 Python-3.11.9 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060, 11910MiB)

Fusing layers... 
YOLOv5x summary: 182 layers, 7246518 parameters, 0 gradients
Adding AutoShape... 


In [8]:
# sau khi load det = torch.hub.load(...)

print("MODEL NAMES =", det.names)     # thường là dict: {0:'face'} hoặc list ['face',...]


MODEL NAMES = {0: 'face'}
